In [47]:
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import pandas as pd
from solvers.catchment_runoff_solver import CatchmentRunoffSolver

storm_rainfall_depth = pd.read_csv("data/PF_Depth_English_PDS.csv")
storm_rainfall_depth = storm_rainfall_depth.drop(["1000"], axis=1)

storm_rainfall_depth = storm_rainfall_depth[storm_rainfall_depth["Storm"].isin(["5-min:", "10-min:", "15-min:", "30-min:", "60-min:"])]

catchment_area=20.4
catchment_impervious_cover_fraction=0.25

swale_a_catchment = 5.0
swale_a_catchment_impervious_cover_fraction = 0.0865

swale_c_catchment = 3.0
swale_c_catchment_impervious_cover_fraction = 0.1275

storm_depths = []
for index, row in storm_rainfall_depth.iterrows():
    for ri in list(storm_rainfall_depth.drop(["Storm"], axis=1).columns):
        depth = row[ri]
        
        rv = CatchmentRunoffSolver(
            catchment_area=catchment_area,
            rainfall_depth=depth,
            impervious_cover_fraction=catchment_impervious_cover_fraction
        ).solve()

        rv_a = CatchmentRunoffSolver(
            catchment_area=swale_a_catchment,
            rainfall_depth=depth,
            impervious_cover_fraction=swale_a_catchment_impervious_cover_fraction
        ).solve()

        rv_c = CatchmentRunoffSolver(
            catchment_area=swale_c_catchment,
            rainfall_depth=depth,
            impervious_cover_fraction=swale_c_catchment_impervious_cover_fraction
        ).solve()
        
        length = row["Storm"].replace(':', '')
        
        storm_depths.append([ri, length, depth, rv, rv_a, rv_c])

storm_depths_df = pd.DataFrame(storm_depths, columns=['recurrence interval (yr)', 'storm length (min)', 'rainfall depth (in)', 'runoff volume (ft^3)', 'runoff volume reaching Swale A (ft^3)', 'runoff volume reaching Swale C (ft^3)'])

storm_depths_df


,recurrence interval (yr),storm length (min),rainfall depth (in),runoff volume (ft^3),runoff volume reaching Swale A (ft^3),runoff volume reaching Swale C (ft^3)
0,1,5-min,0.431,8777.01,1000.13,773.27
1,2,5-min,0.524,10670.89,1215.93,940.12
2,5,5-min,0.668,13603.35,1550.08,1198.48
3,10,5-min,0.795,16189.62,1844.78,1426.33
4,25,5-min,0.977,19895.92,2267.11,1752.86
5,50,5-min,1.120,22808.02,2598.93,2009.42
6,100,5-min,1.280,26066.30,2970.21,2296.48
7,200,5-min,1.460,29731.88,3387.90,2619.43
8,500,5-min,1.700,34619.31,3944.81,3050.02
9,1,10-min,0.684,13929.18,1587.21,1227.18


In [49]:
dates = [
    '2024-04-20 17:51:00.000',
    '2024-04-20 18:51:00.000',
    '2024-04-20 19:51:00.000',
    '2024-04-20 20:51:00.000',
    '2024-04-20 21:51:00.000'
]

depths = [
    0.07,
    0.0,
    0.02,
    0.38,
    0.22
]

local_storms = []
for i, depth in enumerate(depths):
    local = CatchmentRunoffSolver(
        catchment_area=catchment_area,
        rainfall_depth=depth,
        impervious_cover_fraction=catchment_impervious_cover_fraction
    ).solve()

    rv_a = CatchmentRunoffSolver(
        catchment_area=swale_a_catchment,
        rainfall_depth=depth,
        impervious_cover_fraction=swale_a_catchment_impervious_cover_fraction
    ).solve()

    rv_c = CatchmentRunoffSolver(
        catchment_area=swale_c_catchment,
        rainfall_depth=depth,
        impervious_cover_fraction=swale_c_catchment_impervious_cover_fraction
    ).solve()
    
    local_storms.append([dates[i], '60 min', depth, local, rv_a, rv_c])



local_storms_df = pd.DataFrame(local_storms, columns=['datetime', 'storm length (min)', 'rainfall depth (in)', 'runoff volume (ft^3)', 'runoff volume reaching Swale A (ft^3)', 'runoff volume reaching Swale C (ft^3)'])
local_storms_df

,datetime,storm length (min),rainfall depth (in),runoff volume (ft^3),runoff volume reaching Swale A (ft^3),runoff volume reaching Swale C (ft^3)
0,2024-04-20 17:51:00.000,60 min,0.07,1425.50,162.43,125.59
1,2024-04-20 18:51:00.000,60 min,0.00,0.00,0.00,0.00
2,2024-04-20 19:51:00.000,60 min,0.02,407.29,46.41,35.88
3,2024-04-20 20:51:00.000,60 min,0.38,7738.43,881.78,681.77
4,2024-04-20 21:51:00.000,60 min,0.22,4480.15,510.51,394.71


In [50]:
from solvers import ChannelType
from solvers.channel_discharge_solver import ChannelDischargeSolver
from solvers.volume_solver import VolumeSolver

q_a_solve = ChannelDischargeSolver(
    channel_type=ChannelType.Trapezoid,
    manning_roughness_coefficient=0.08,
    channel_slope=0.2,
    bottom_width_channel=1.513888889,
    depth_flow=0.805555556,
    side_slope=0.032754349
)

q_a = q_a_solve.solve()

v_a = VolumeSolver(
    channel_type=ChannelType.Trapezoid,
    channel_length=160,
    bottom_width_channel=1.513888889,
    top_width_channel=4.479166667,
    depth_flow=0.805555556
).solve()

q_c_solve = ChannelDischargeSolver(
    channel_type=ChannelType.Trapezoid,
    manning_roughness_coefficient=0.04,
    channel_slope=0.2,
    bottom_width_channel=1.5625,
    depth_flow=1.3125,
    side_slope=0.014176131
)
q_c = q_c_solve.solve()
v_c = VolumeSolver(
    channel_type=ChannelType.Trapezoid,
    channel_length=58,
    bottom_width_channel=1.5625,
    top_width_channel=4.416666667,
    depth_flow=1.3125
).solve()

v_b1 = VolumeSolver(
    channel_type=ChannelType.Rectangle,
    channel_length=19,
    top_width_channel=7.354166667,
    depth_flow=0.805555556
).solve()

v_b2 = VolumeSolver(
    channel_type=ChannelType.Rectangle,
    channel_length=14,
    top_width_channel=4.861111111,
    depth_flow=1.111111111
).solve()

v_c_summed = round(sum([v_b1, v_b2]), 2)

In [51]:
from solvers.ponding_soil_infiltration_solver import PondingSoilInfiltrationSolver

a_s_d, a_s_w, a_s_r, a_s_ph, a_s_v = PondingSoilInfiltrationSolver(
    wetted_perimeter = q_a_solve.wp,
    flow_volume = v_a,
    channel_length = 160,
    soil_infiltration_rate = 0.3,
    soil_depth = 6.6,
    soil_water_capacity_dry = 0.58,
    soil_water_capacity_wet = 0.14
).solve()

c_s_d, c_s_w, c_s_r, c_s_ph, c_s_v = PondingSoilInfiltrationSolver(
    wetted_perimeter = q_c_solve.wp,
    flow_volume = v_c,
    channel_length = 58,
    soil_infiltration_rate = 0.3,
    soil_depth = 6.6,
    soil_water_capacity_dry = 0.58,
    soil_water_capacity_wet = 0.14
).solve()

from solvers.catchment_rainfall_depth_solver import CatchmentRainfallDepthSolver

rd_a, rd_asd, rd_asw = CatchmentRainfallDepthSolver(
    catchment_area=swale_a_catchment,
    runoff_volume=v_a,
    impervious_cover_fraction=swale_a_catchment_impervious_cover_fraction,
    total_capacity_with_dry_soil = a_s_w,
    total_capacity_with_wet_soil = a_s_r
).solve()

rd_c, rd_csd, rd_csw = CatchmentRainfallDepthSolver(
    catchment_area=swale_c_catchment,
    runoff_volume=v_c,
    impervious_cover_fraction=swale_c_catchment_impervious_cover_fraction,
    total_capacity_with_dry_soil = c_s_w,
    total_capacity_with_wet_soil = c_s_r
).solve()



In [52]:
swale_calcs = []
swale_calcs.append(    ['A',
                        q_a,
                        v_a,
                        rd_a,
                        0,
                        round(v_a / q_a, 2)
                        ])
swale_calcs.append(    ['C',
                        q_c,
                        v_c,
                        rd_c,
                        v_c_summed,
                        round(v_c / q_c, 2)
                        ])
swale_df = pd.DataFrame(swale_calcs, columns=[
        'swale name', 
        'peak discharge capacity (ft^3/s)', 
        'volume capacity (ft^3)',
        'rainfall depth to reach volume capacity (ft^3)',
        'potential basin capacity (ft^3)', 
        'time to fill (s)'
    ])
swale_df

,swale name,peak discharge capacity (ft^3/s),volume capacity (ft^3),rainfall depth to reach volume capacity (ft^3),potential basin capacity (ft^3),time to fill (s)
0,A,5.58,386.22,0.17,0.00,69.22
1,C,21.65,227.58,0.13,188.18,10.51


In [54]:
swale_soil_calcs = []
swale_soil_calcs.append(    ['A',
                        a_s_d,
                        a_s_w,
                        a_s_r,
                        a_s_ph,
                        a_s_v,
                        sum([v_a, a_s_w]),
                        rd_asd,
                        sum([v_a, a_s_r]),
                        rd_asw
                        ])
swale_soil_calcs.append(    ['C',
                        c_s_d,
                        c_s_w,
                        c_s_r,
                        c_s_ph,
                        c_s_v,
                        sum([v_c, c_s_w]),
                        rd_csd,
                        sum([v_c, c_s_r]),
                        rd_csw
                        ])
swale_soil_df = pd.DataFrame(swale_soil_calcs, columns=[
    'swale name',
    "soil volume (ft^3)",
    "soil dry capacity (ft^3)",
    "soil wet capacity (ft^3)",
    'ft^3/hr infiltrated',
    'infiltration time (hr)',
    'total capacity w/soil dry (ft^3)',
    'rainfall depth to reach volume capacity w/soil dry (in)',
    'total capacity w/soil wet (ft^3)',
    'rainfall depth to reach volume capacity w/soil wet (in)'
])
swale_soil_df

,swale name,soil volume (ft^3),soil dry capacity (ft^3),soil wet capacity (ft^3),ft^3/hr infiltrated,infiltration time (hr),total capacity w/soil dry (ft^3),rainfall depth to reach volume capacity w/soil dry (in),total capacity w/soil wet (ft^3),rainfall depth to reach volume capacity w/soil wet (in)
0,A,3300.91,1914.53,462.13,12.503456,30.89,2300.75,0.83,848.35,0.20
1,C,1603.08,929.78,224.43,6.072257,37.48,1157.36,0.52,452.01,0.13
